In [9]:
# Python 3.6, Numpy 1.16.2, OpenCV, scikit-image, tensorflow 1.3
import tensorflow as tf
import vgg19_fastmal as vgg19
import utils
import argparse 
import os
import csv
import numpy as np
import pandas as pd

def majority_voting(votes):
    return max(votes,key=votes.count)

In [2]:
# make the training and testing csv file
with open("/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle_slides_new_march.csv", newline='') as csvfile:
    data = np.array(list(csv.reader(csvfile)))
# print(data)
training_list =[]
training_labels = []
testing_list = []
testing_labels = []
csv_slide_ids = data[:,0]
csv_labels = np.array(data[:,1]).astype(np.uint8)
test_list = os.listdir("/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle3-edofed")
for num,i in enumerate(csv_slide_ids):
    if i not in test_list:
        training_list.append(i)
        training_labels.append(csv_labels[num])
    else:
        testing_list.append(i)
        testing_labels.append(csv_labels[num])
df_train = pd.DataFrame({'slide_name':training_list,'label':training_labels})
df_test = pd.DataFrame({'slide_name':testing_list,'label':testing_labels})
df_train.to_csv("/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/train.csv",index=False)
df_test.to_csv("/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/test.csv",index=False)
        

In [3]:
# read the training csv file as dataframe
df_train = pd.read_csv("/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/train.csv")

csv_slide_ids = df_train['slide_name'].tolist()
print(len(csv_slide_ids))
csv_labels =np.array(df_train['label']).astype(np.uint8)
onehot_labels = np.zeros(shape=(csv_labels.shape[0], 2), dtype=np.float32)
onehot_labels[csv_labels==0,0]=1
onehot_labels[csv_labels==1,1]=1

94


In [4]:
# read the testing csv file as dataframe
df_test = pd.read_csv("/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/test.csv")
test_csv_slide_ids = df_test['slide_name'].tolist()
print(len(test_csv_slide_ids))
test_csv_labels =np.array(df_test['label']).astype(np.uint8)

68


In [5]:
#load the data path
dataset = "/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed"
dataset2 = None
test_dir = "/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle3-edofed"
test_dir2 = None

selected_slides = []
selected_labels=[]
selected_onehot_labels = []

test_slides = []
test_labels=[]
test_onehot_labels = []

subdirs = [x[0] for x in os.walk(dataset)][1:]  
subdirs2=[]
if dataset2 is not None:   
    subdirs2 = [x[0] for x in os.walk(dataset2)][1:]     

subdirsAll=subdirs+subdirs2
test_subdirs = [y[0] for y in os.walk(test_dir)]#[1:]
#print(test_subdirs)
test_subdirs2=[]
if test_dir2 is not None:   
    test_subdirs2 = [y[0] for y in os.walk(test_dir2)]#[1:]
     
test_subdirsAll=test_subdirs+test_subdirs2
print(subdirsAll[:10])
print(test_subdirsAll[:10])

['/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/010319-05r1-S-A-S1-20190816160305', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/280120-29', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/311017-03-S-A-S1-20190426160015', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/311017-05-S-A-S1-20190501131656', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/080119-48', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/290318-02', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/080119-07', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/241017-21-S-A-S1-20190704170416', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/020919-01', '/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed/150218-01-S

In [6]:
for subdir in subdirsAll: 
    predicted_rois = []
    slide_path = subdir
    slide_id = slide_path.split(os.path.sep)[-1]
    if slide_id in csv_slide_ids:
        sel_id = list(csv_slide_ids).index(slide_id)
        print(slide_id, ' has label ', csv_labels[sel_id], ' or ', onehot_labels[sel_id,:])
        selected_slides.append(slide_path) 
        selected_onehot_labels.append(onehot_labels[sel_id,:])
        selected_labels.append(csv_labels[sel_id])
for tsubdir in test_subdirsAll: 
    #predicted_rois = []
    slide_path = tsubdir
    slide_id = slide_path.split(os.path.sep)[-1]
    #print(slide_id)
    if slide_id in test_csv_slide_ids:
        sel_id = list(test_csv_slide_ids).index(slide_id)
        print(slide_id, ' has label ', test_csv_labels[sel_id])
        test_slides.append(slide_path) 
        #test_onehot_labels.append(onehot_labels[sel_id,:])
        test_labels.append(test_csv_labels[sel_id])    


selected_labels=np.array(selected_labels)
positive_ids = np.nonzero(selected_labels)[0]
negative_ids = np.nonzero(1-selected_labels)[0]


print("Negative train:", np.sum(1-selected_labels))
print("Positive train:", np.sum(selected_labels))

print("Negative test:", np.sum(1-np.array(test_labels)))
print("Positive test:", np.sum(np.array(test_labels)))

010319-05r1-S-A-S1-20190816160305  has label  0  or  [1. 0.]
311017-03-S-A-S1-20190426160015  has label  0  or  [1. 0.]
080119-48  has label  0  or  [1. 0.]
290318-02  has label  1  or  [0. 1.]
080119-07  has label  1  or  [0. 1.]
020919-01  has label  0  or  [1. 0.]
150218-01-S-A-S1-20190424164843  has label  1  or  [0. 1.]
171017-21-S-A-S1-20190624162525  has label  0  or  [1. 0.]
260618-44  has label  0  or  [1. 0.]
280120-05  has label  1  or  [0. 1.]
101017-16-S-A-S1-20190603132416  has label  0  or  [1. 0.]
280120-37  has label  1  or  [0. 1.]
311017-06-S-A-S1-20190501134757  has label  0  or  [1. 0.]
280120-16  has label  1  or  [0. 1.]
080119-45  has label  1  or  [0. 1.]
101017-07r3  has label  1  or  [0. 1.]
101017-04-S-A-S1-20190510101904  has label  0  or  [1. 0.]
280120-57  has label  1  or  [0. 1.]
030919-07  has label  1  or  [0. 1.]
080119-25  has label  0  or  [1. 0.]
020919-08  has label  0  or  [1. 0.]
101017-09-S-A-S1-20190510123456  has label  1  or  [0. 1.]
101017

In [12]:
IMSIZE = 64
num_labels=2
num_steps = 0
batch_size= 1
rpt_interval=100
min_nb_images = 1
save_dir = "../output/"
dataset = "/home/weber50432/AML_image_processing/FASt-MAL-MOFF-master/data/sickle-edofed"
with tf.Session() as sess:

    images = tf.placeholder(tf.float32, [None, IMSIZE, IMSIZE, 3])
    true_out = tf.placeholder(tf.float32, [None, 2])
    train_mode = tf.placeholder(tf.bool)

    vgg = vgg19.Vgg19("../model/vgg19.npy", imsize=IMSIZE)
    vgg.build_avg_pool(images, train_mode=train_mode)
    focal_loss = tf.reduce_mean(
     tf.nn.softmax_cross_entropy_with_logits(logits=vgg.new_fc8, labels=true_out)) 
    train = tf.train.GradientDescentOptimizer(0.0003).minimize(focal_loss)
    sess.run(tf.global_variables_initializer())
    step_pos=0
    step_neg=0
    for step in range(1, num_steps+1):
      try:  
          offset = (step * batch_size) % (len(selected_onehot_labels) - batch_size)
          
          if step%2==0:
              sl_id = positive_ids[step_pos % (positive_ids.shape[0])]
              step_pos=step_pos+1
          else:
              sl_id = negative_ids[step_neg % (negative_ids.shape[0])]
              step_neg=step_neg+1

          slides=utils.load_folder_random(selected_slides[sl_id], max_no_img=200, crop_size=IMSIZE)
          if len(slides)>min_nb_images:
              slides = np.array(slides)
              labels = np.reshape(selected_onehot_labels[sl_id], (batch_size, num_labels))
              _,l = sess.run([train, focal_loss], feed_dict={images: slides, true_out: labels, train_mode: True})
          if (step % rpt_interval == 0):
              print('Minibatch loss at step %d: %f' % (step, l))     
          if (step % 5000 == 0):   
            save_path = os.path.join(save_dir,dataset.split(os.path.sep)[-3]+str(step)+"sickle_max_pool_ce_vgg19_model.npy")
            vgg.save_npy(sess, save_path)
            print("Model saved in file: %s" % save_path)    
      except IOError as e:
        print('Could not read:', selected_slides[offset], ':', e, '- it\'s ok, skipping.')        
# test classification again, should have a higher probability about tiger
    prediction_csv = dataset.split(os.path.sep)[-2]+'_mv_all_normal_max_pool2.csv' 
    header=['Slide-Id', 'True', 'Predicted']
    predictionFile= open(os.path.join("../output/", prediction_csv),'w')  
    wr = csv.writer(predictionFile, dialect='excel')
    wr.writerow(header)    
    predicted_classif = np.zeros(len(test_slides))
    true_classif = np.zeros(len(test_slides))
    for tt in range(len(test_slides)):
        
        classify_voting=[]
        probs=[]
        for run in range(1):
        #tslides = utils.load_folder(test_slides[tt], crop_size=64)
            tslides=utils.load_folder_random(test_slides[tt], max_no_img=4, crop_size=IMSIZE)
        
            tslide_id = test_slides[tt].split(os.path.sep)[-1]
            malaria_classif=0
            if len(tslides)>min_nb_images:
                tslides = np.array(tslides)

                prob = sess.run(vgg.new_prob, feed_dict={images: tslides, train_mode: False})
                #print(prob)
                classify_voting.append(np.argmax(prob))
                probs.append(prob[0,1])
        sel_id = list(test_csv_slide_ids).index(tslide_id)
        true_classif[tt] = test_csv_labels[sel_id]
        predicted_classif[tt]=majority_voting(classify_voting)
        #predicted_classif[tt]=max(classify_voting)
        wr.writerow([tslide_id, true_classif[tt], predicted_classif[tt], np.mean(probs)])  
        print(tslide_id, true_classif[tt], predicted_classif[tt], np.mean(probs))
        
        
        
    

    overall_accuracy=np.mean(true_classif==predicted_classif)    
    print('Overall accuracy', overall_accuracy)
    true_pos = true_classif[true_classif==1]
    pred_pos = predicted_classif[true_classif==1]
    positive_accuracy=np.mean(true_pos==pred_pos)
    print('Postive accuracy', positive_accuracy)
    true_neg = true_classif[true_classif==0]    
    pred_neg = predicted_classif[true_classif==0]
    negative_accuracy=np.mean(true_neg==pred_neg)
    print('Negative accuracy', negative_accuracy)   

ValueError: Dimension size must be evenly divisible by 8192 but is 2048 for 'new_fc6_4/Reshape' (op: 'Reshape') with input shapes: [2,2,512], [2] and with input tensors computed as partial shapes: input[1] = [?,8192].